# Normal Resnet-18

In [ ]:
!pip install ptflops

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 117.3 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 34.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 14.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 95.8 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5

In [ ]:
import time
import csv
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as T
from torch.utils.data import DataLoader
from torchvision.models import resnet18, ResNet18_Weights

from ptflops import get_model_complexity_info
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

Device: cuda


In [ ]:
# 1. CIFAR-10 DATASET (224x224)

transform_train = T.Compose([
    T.Resize((224, 224)),
    T.RandomHorizontalFlip(),
    T.ToTensor(),
])

transform_test = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
])

trainset = torchvision.datasets.CIFAR10(
    root="./data",
    train=True,
    download=True,
    transform=transform_train,
)
testset = torchvision.datasets.CIFAR10(
    root="./data",
    train=False,
    download=True,
    transform=transform_test,
)

trainloader = DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)
testloader = DataLoader(testset, batch_size=128, shuffle=False, num_workers=2)

classes = trainset.classes
num_classes = 10

In [ ]:
# 2. BUILD BASE RESNET-18

model = resnet18(
    weights=ResNet18_Weights.IMAGENET1K_V1,
)

model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

num_epochs = 20

In [ ]:
# 3. TRAIN & TEST FUNCTIONS

def train_one_epoch(model, loader):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for imgs, labels in loader:
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = outputs.max(1)
        total += labels.size(0)
        correct += preds.eq(labels).sum().item()

    epoch_loss = running_loss / len(loader)
    epoch_acc = 100.0 * correct / total
    return epoch_loss, epoch_acc


def evaluate(model, loader):
    model.eval()
    correct = 0
    total = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for imgs, labels in loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            _, preds = outputs.max(1)
            total += labels.size(0)
            correct += preds.eq(labels).sum().item()
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    acc = 100.0 * correct / total
    return acc, np.array(all_labels), np.array(all_preds)

In [ ]:
# 4. TRAINING LOOP

train_losses, train_accs, test_accs = [], [], []
train_start = time.time()

for epoch in range(num_epochs):
    t0 = time.time()
    loss, train_acc = train_one_epoch(model, trainloader)
    test_acc, _, _ = evaluate(model, testloader)
    t1 = time.time()

    train_losses.append(loss)
    train_accs.append(train_acc)
    test_accs.append(test_acc)

    print(f"Epoch {epoch+1}/{num_epochs} | "
          f"Loss={loss:.4f} | "
          f"TrainAcc={train_acc:.2f}% | "
          f"TestAcc={test_acc:.2f}% | "
          f"EpochTime={t1 - t0:.2f}s")

train_end = time.time()
total_train_time = train_end - train_start
print(f"\nTotal training time: {total_train_time:.2f} seconds "
      f"({total_train_time/60:.2f} min)")

# Final accuracies
final_train_acc = train_accs[-1]
final_test_acc, all_labels, all_preds = evaluate(model, testloader)
print(f"Final Train Acc: {final_train_acc:.2f}%")
print(f"Final Test Acc:  {final_test_acc:.2f}%")

Epoch 1/20 | Loss=0.3350 | TrainAcc=89.25% | TestAcc=92.83% | EpochTime=95.49s
Epoch 2/20 | Loss=0.1197 | TrainAcc=96.20% | TestAcc=93.57% | EpochTime=95.17s
Epoch 3/20 | Loss=0.0664 | TrainAcc=97.97% | TestAcc=94.38% | EpochTime=95.25s
Epoch 4/20 | Loss=0.0463 | TrainAcc=98.64% | TestAcc=94.87% | EpochTime=94.68s
Epoch 5/20 | Loss=0.0314 | TrainAcc=99.04% | TestAcc=94.86% | EpochTime=94.91s
Epoch 6/20 | Loss=0.0246 | TrainAcc=99.28% | TestAcc=94.60% | EpochTime=95.24s
Epoch 7/20 | Loss=0.0226 | TrainAcc=99.32% | TestAcc=95.00% | EpochTime=94.76s
Epoch 8/20 | Loss=0.0201 | TrainAcc=99.35% | TestAcc=94.10% | EpochTime=95.03s
Epoch 9/20 | Loss=0.0166 | TrainAcc=99.49% | TestAcc=94.73% | EpochTime=97.28s
Epoch 10/20 | Loss=0.0191 | TrainAcc=99.40% | TestAcc=94.53% | EpochTime=95.18s
Epoch 11/20 | Loss=0.0176 | TrainAcc=99.44% | TestAcc=94.75% | EpochTime=95.46s
Epoch 12/20 | Loss=0.0168 | TrainAcc=99.43% | TestAcc=95.14% | EpochTime=95.17s
Epoch 13/20 | Loss=0.0147 | TrainAcc=99.55% | Tes

In [ ]:
# 6. FLOPs & PARAMS (ptflops)

with torch.no_grad():
    macs, params = get_model_complexity_info(
        model,
        input_res=(3, 224, 224),
        as_strings=False,
        print_per_layer_stat=False,
        verbose=False,
    )
flops = macs  # treat MACs as FLOPs/MADs

print(f"Params: {int(params)}")
print(f"FLOPs:  {flops:.3g}")

Params: 11181642
FLOPs:  1.82e+09


In [ ]:
# 7. INFERENCE & TRAIN RUNTIME

dummy_x = torch.randn(1, 3, 224, 224).to(device)

# Inference runtime
model.eval()
if device == "cuda":
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats()
    torch.cuda.synchronize()

with torch.no_grad():
    for _ in range(10):
        _ = model(dummy_x)

if device == "cuda":
    torch.cuda.synchronize()
t0 = time.time()
with torch.no_grad():
    for _ in range(100):
        _ = model(dummy_x)
if device == "cuda":
    torch.cuda.synchronize()
t1 = time.time()

infer_ms = (t1 - t0) / 100.0 * 1000.0
if device == "cuda":
    mem_infer_mb = torch.cuda.max_memory_allocated() / (1024**2)
else:
    mem_infer_mb = 0.0

# Training runtime (single-batch loop)
model.train()
dummy_label = torch.randint(0, num_classes, (1,), device=device)
optimizer = optim.SGD(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

if device == "cuda":
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats()
    torch.cuda.synchronize()

for _ in range(10):
    optimizer.zero_grad()
    out = model(dummy_x)
    loss = criterion(out, dummy_label)
    loss.backward()
    optimizer.step()

if device == "cuda":
    torch.cuda.synchronize()
t0 = time.time()
for _ in range(100):
    optimizer.zero_grad()
    out = model(dummy_x)
    loss = criterion(out, dummy_label)
    loss.backward()
    optimizer.step()
if device == "cuda":
    torch.cuda.synchronize()
t1 = time.time()

train_ms = (t1 - t0) / 100.0 * 1000.0
if device == "cuda":
    mem_train_mb = torch.cuda.max_memory_allocated() / (1024**2)
else:
    mem_train_mb = 0.0

mem_mb = max(mem_infer_mb, mem_train_mb)

print(f"\nInference time per image: {infer_ms:.3f} ms")
print(f"Train step time per image: {train_ms:.3f} ms")
print(f"Peak GPU memory: {mem_mb:.2f} MB")


Inference time per image: 2.232 ms
Train step time per image: 7.227 ms
Peak GPU memory: 214.67 MB


In [ ]:
# 8. SAVE MODEL & METRICS CSV

torch.save(model.state_dict(), "/kaggle/working/resnet18_cifar10.pth")

csv_path = "/kaggle/working/resnet18_full_metrics.csv"
with open(csv_path, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow([
        "Model", "Params", "FLOPs",
        "Infer_ms", "Train_ms",
        "Memory_MB", "TotalTrainTimeSec",
        "FinalTrainAcc", "FinalTestAcc"
    ])
    writer.writerow([
        "ResNet18",
        int(params),
        flops,
        infer_ms,
        train_ms,
        mem_mb,
        total_train_time,
        final_train_acc,
        final_test_acc,
    ])

print(f"\nMetrics CSV saved to: {csv_path}")
print("Baseline ResNet18 experiment complete.")


Metrics CSV saved to: /kaggle/working/resnet18_full_metrics.csv
Baseline ResNet18 experiment complete.


# Oriented 1D ResNet

In [ ]:
# import required libraries
import time
import csv
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as T
from torch.utils.data import DataLoader
from torchvision.models import resnet18, ResNet18_Weights

from torchvision.transforms.functional import rotate
from torchvision.transforms import InterpolationMode

from ptflops import get_model_complexity_info
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

Device: cuda


In [ ]:
# 1. CIFAR-10 DATASET (224x224)

transform_train = T.Compose([
    T.Resize((224, 224)),
    T.RandomHorizontalFlip(),
    T.ToTensor(),
])

transform_test = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
])

trainset = torchvision.datasets.CIFAR10(
    root="./data",
    train=True,
    download=True,
    transform=transform_train,
)
testset = torchvision.datasets.CIFAR10(
    root="./data",
    train=False,
    download=True,
    transform=transform_test,
)

trainloader = DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)
testloader = DataLoader(testset, batch_size=128, shuffle=False, num_workers=2)

classes = trainset.classes
num_classes = 10

In [ ]:
# 2. Oriented 1D Conv Layer

class Oriented1DConv2d(nn.Module):
    def __init__(
        self,
        in_channels,
        out_channels,
        kernel_size=7,
        num_angles=8,
        stride=1,
        bias=True,
    ):
        super().__init__()
        assert kernel_size % 2 == 1
        assert stride == 1

        self.num_angles = num_angles
        self.angles = [i * 180.0 / num_angles for i in range(num_angles)]

        pad = kernel_size // 2
        self.conv1d = nn.Conv2d(
            in_channels,
            out_channels,
            kernel_size=(1, kernel_size),
            stride=(1, 1),
            padding=(0, pad),
            bias=bias,
        )

    def forward(self, x):
        outs = []
        for ang in self.angles:
            xr = rotate(x, angle=ang, interpolation=InterpolationMode.BILINEAR)
            y = self.conv1d(xr)
            y = rotate(y, angle=-ang, interpolation=InterpolationMode.BILINEAR)
            outs.append(y)
        return sum(outs) / self.num_angles

In [ ]:
# 3. Build Oriented ResNet-18

def make_oriented_resnet18(
    num_classes=10,
    kernel_size=7,
    num_angles=8,
):
    base = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)

    def replace(layer):
        for block in layer:
            if hasattr(block, "conv2"):
                old = block.conv2
                if isinstance(old, nn.Conv2d) and old.kernel_size == (3, 3):
                    block.conv2 = Oriented1DConv2d(
                        in_channels=old.in_channels,
                        out_channels=old.out_channels,
                        kernel_size=kernel_size,
                        num_angles=num_angles,
                        stride=1,
                        bias=(old.bias is not None),
                    )

    replace(base.layer1)
    replace(base.layer2)
    replace(base.layer3)
    replace(base.layer4)

    base.fc = nn.Linear(base.fc.in_features, num_classes)
    return base.to(device)


model = make_oriented_resnet18(num_classes=num_classes, kernel_size=7, num_angles=8)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

num_epochs = 20

In [ ]:
# 4. TRAIN & TEST FUNCTIONS

def train_one_epoch(model, loader):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for imgs, labels in loader:
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = outputs.max(1)
        total += labels.size(0)
        correct += preds.eq(labels).sum().item()

    epoch_loss = running_loss / len(loader)
    epoch_acc = 100.0 * correct / total
    return epoch_loss, epoch_acc


def evaluate(model, loader):
    model.eval()
    correct = 0
    total = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for imgs, labels in loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            _, preds = outputs.max(1)
            total += labels.size(0)
            correct += preds.eq(labels).sum().item()
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    acc = 100.0 * correct / total
    return acc, np.array(all_labels), np.array(all_preds)

In [ ]:
# 5. TRAINING LOOP

train_losses, train_accs, test_accs = [], [], []
train_start = time.time()

for epoch in range(num_epochs):
    t0 = time.time()
    loss, train_acc = train_one_epoch(model, trainloader)
    test_acc, _, _ = evaluate(model, testloader)
    t1 = time.time()

    train_losses.append(loss)
    train_accs.append(train_acc)
    test_accs.append(test_acc)

    print(f"Epoch {epoch+1}/{num_epochs} | "
          f"Loss={loss:.4f} | "
          f"TrainAcc={train_acc:.2f}% | "
          f"TestAcc={test_acc:.2f}% | "
          f"EpochTime={t1 - t0:.2f}s")

train_end = time.time()
total_train_time = train_end - train_start
print(f"\nTotal training time: {total_train_time:.2f} seconds "
      f"({total_train_time/60:.2f} min)")

final_train_acc = train_accs[-1]
final_test_acc, all_labels, all_preds = evaluate(model, testloader)
print(f"Final Train Acc: {final_train_acc:.2f}%")
print(f"Final Test Acc:  {final_test_acc:.2f}%")

Epoch 1/20 | Loss=1.2242 | TrainAcc=55.89% | TestAcc=59.07% | EpochTime=516.84s
Epoch 2/20 | Loss=0.7704 | TrainAcc=72.81% | TestAcc=72.08% | EpochTime=516.36s
Epoch 3/20 | Loss=0.5911 | TrainAcc=79.36% | TestAcc=72.86% | EpochTime=516.01s
Epoch 4/20 | Loss=0.4732 | TrainAcc=83.53% | TestAcc=80.07% | EpochTime=516.05s
Epoch 5/20 | Loss=0.3910 | TrainAcc=86.33% | TestAcc=79.90% | EpochTime=516.69s
Epoch 6/20 | Loss=0.3252 | TrainAcc=88.77% | TestAcc=79.19% | EpochTime=516.29s
Epoch 7/20 | Loss=0.2730 | TrainAcc=90.49% | TestAcc=81.52% | EpochTime=516.72s
Epoch 8/20 | Loss=0.2301 | TrainAcc=91.91% | TestAcc=82.16% | EpochTime=516.45s
Epoch 9/20 | Loss=0.2009 | TrainAcc=92.82% | TestAcc=84.16% | EpochTime=516.66s
Epoch 10/20 | Loss=0.1625 | TrainAcc=94.34% | TestAcc=82.48% | EpochTime=517.78s
Epoch 11/20 | Loss=0.1367 | TrainAcc=95.16% | TestAcc=84.58% | EpochTime=517.06s
Epoch 12/20 | Loss=0.1211 | TrainAcc=95.74% | TestAcc=85.28% | EpochTime=516.80s
Epoch 13/20 | Loss=0.1064 | TrainAcc=

In [ ]:
# 7. FLOPs & PARAMS

with torch.no_grad():
    macs, params = get_model_complexity_info(
        model,
        input_res=(3, 224, 224),
        as_strings=False,
        print_per_layer_stat=False,
        verbose=False,
    )
flops = macs
print(f"Params: {int(params)}")
print(f"FLOPs:  {flops:.3g}")

Params: 9789002
FLOPs:  6.66e+09


In [ ]:
# 8. INFERENCE & TRAIN RUNTIME

dummy_x = torch.randn(1, 3, 224, 224).to(device)

# Inference runtime
model.eval()
if device == "cuda":
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats()
    torch.cuda.synchronize()

with torch.no_grad():
    for _ in range(10):
        _ = model(dummy_x)

if device == "cuda":
    torch.cuda.synchronize()
t0 = time.time()
with torch.no_grad():
    for _ in range(100):
        _ = model(dummy_x)
if device == "cuda":
    torch.cuda.synchronize()
t1 = time.time()

infer_ms = (t1 - t0) / 100.0 * 1000.0
if device == "cuda":
    mem_infer_mb = torch.cuda.max_memory_allocated() / (1024**2)
else:
    mem_infer_mb = 0.0

# Training runtime
model.train()
dummy_label = torch.randint(0, num_classes, (1,), device=device)
optimizer = optim.SGD(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

if device == "cuda":
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats()
    torch.cuda.synchronize()

for _ in range(10):
    optimizer.zero_grad()
    out = model(dummy_x)
    loss = criterion(out, dummy_label)
    loss.backward()
    optimizer.step()

if device == "cuda":
    torch.cuda.synchronize()
t0 = time.time()
for _ in range(100):
    optimizer.zero_grad()
    out = model(dummy_x)
    loss = criterion(out, dummy_label)
    loss.backward()
    optimizer.step()
if device == "cuda":
    torch.cuda.synchronize()
t1 = time.time()

train_ms = (t1 - t0) / 100.0 * 1000.0
if device == "cuda":
    mem_train_mb = torch.cuda.max_memory_allocated() / (1024**2)
else:
    mem_train_mb = 0.0

mem_mb = max(mem_infer_mb, mem_train_mb)

print(f"\nInference time per image: {infer_ms:.3f} ms")
print(f"Train step time per image: {train_ms:.3f} ms")
print(f"Peak GPU memory: {mem_mb:.2f} MB")


Inference time per image: 54.666 ms
Train step time per image: 114.454 ms
Peak GPU memory: 290.07 MB


In [ ]:
# 9. SAVE MODEL & METRICS CSV

torch.save(model.state_dict(), "/kaggle/working/oriented_resnet18_cifar10.pth")

csv_path = "/kaggle/working/oriented_resnet18_full_metrics.csv"
with open(csv_path, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow([
        "Model", "Params", "FLOPs",
        "Infer_ms", "Train_ms",
        "Memory_MB", "TotalTrainTimeSec",
        "FinalTrainAcc", "FinalTestAcc"
    ])
    writer.writerow([
        "OrientedResNet18",
        int(params),
        flops,
        infer_ms,
        train_ms,
        mem_mb,
        total_train_time,
        final_train_acc,
        final_test_acc,
    ])

print(f"\nMetrics CSV saved to: {csv_path}")
print("Oriented-ResNet18 experiment complete.")


Metrics CSV saved to: /kaggle/working/oriented_resnet18_full_metrics.csv
Oriented-ResNet18 experiment complete.


# Compare the models

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


# Load Both Metric CSV Files

resnet_csv = "/kaggle/working/resnet18_full_metrics.csv"
oriented_csv = "/kaggle/working/oriented_resnet18_full_metrics.csv"

df_res = pd.read_csv(resnet_csv)
df_ori = pd.read_csv(oriented_csv)

df_res["Type"] = "ResNet18"
df_ori["Type"] = "OrientedResNet18"


# Combine Two Tables

df = pd.concat([df_res, df_ori], ignore_index=True)

print("\n===== Combined Metrics Table =====\n")
print(df)


===== Combined Metrics Table =====

              Model    Params       FLOPs   Infer_ms    Train_ms   Memory_MB  \
0          ResNet18  11181642  1824805898   2.231948    7.227046  214.671875   
1  OrientedResNet18   9789002  6655346634  54.666362  114.453979  290.074219   

   TotalTrainTimeSec  FinalTrainAcc  FinalTestAcc              Type  
0        1904.253412         99.562         95.32          ResNet18  
1       10333.889185         97.882         86.75  OrientedResNet18  


In [ ]:
# Compute Ratios

baseline = df[df["Model"] == "ResNet18"].iloc[0]
oriented = df[df["Model"] == "OrientedResNet18"].iloc[0]

InferenceRatio = oriented["Infer_ms"] / baseline["Infer_ms"]
TrainRatio = oriented["Train_ms"] / baseline["Train_ms"]
FLOPsRatio = oriented["FLOPs"] / baseline["FLOPs"]
MemoryRatio = oriented["Memory_MB"] / baseline["Memory_MB"]
Efficiency = InferenceRatio / FLOPsRatio

print("\n===== RATIOS (Oriented / Baseline) =====")
print(f"Inference Runtime Ratio: {InferenceRatio:.4f}")
print(f"Training Runtime Ratio : {TrainRatio:.4f}")
print(f"FLOPs Ratio           : {FLOPsRatio:.4f}")
print(f"Memory Ratio          : {MemoryRatio:.4f}")
print(f"Efficiency            : {Efficiency:.4f}")


===== RATIOS (Oriented / Baseline) =====
Inference Runtime Ratio: 24.4927
Training Runtime Ratio : 15.8369
FLOPs Ratio           : 3.6472
Memory Ratio          : 1.3512
Efficiency            : 6.7156


In [ ]:
# Save comparison CSV

comparison_csv_path = "/kaggle/working/model_comparison.csv"

df_compare = pd.DataFrame({
    "Metric": [
        "Params", "FLOPs", "Inference(ms)", "Training(ms)",
        "Memory_MB", "Efficiency", "InferenceRatio", "FLOPsRatio"
    ],
    "ResNet18": [
        baseline["Params"], baseline["FLOPs"], baseline["Infer_ms"],
        baseline["Train_ms"], baseline["Memory_MB"], None, None, None
    ],
    "OrientedResNet18": [
        oriented["Params"], oriented["FLOPs"], oriented["Infer_ms"],
        oriented["Train_ms"], oriented["Memory_MB"], Efficiency,
        InferenceRatio, FLOPsRatio
    ]
})

df_compare.to_csv(comparison_csv_path, index=False)
print(f"\nComparison CSV saved to: {comparison_csv_path}")


Comparison CSV saved to: /kaggle/working/model_comparison.csv


# Visualization of the metrics

In [ ]:
# FLOPs Comparison
plt.figure(figsize=(6,4))
sns.barplot(x="Type", y="FLOPs", data=df)
plt.title("FLOPs Comparison")
plt.savefig("/kaggle/working/flops_comparison.png")
plt.close()

In [ ]:
# Inference Time Comparison
plt.figure(figsize=(6,4))
sns.barplot(x="Type", y="Infer_ms", data=df)
plt.title("Inference Time (ms) Comparison")
plt.savefig("/kaggle/working/inference_time_comparison.png")
plt.close()

In [ ]:
# Training Time Comparison
plt.figure(figsize=(6,4))
sns.barplot(x="Type", y="Train_ms", data=df)
plt.title("Training Time (ms) Comparison")
plt.savefig("/kaggle/working/train_time_comparison.png")
plt.close()

In [ ]:
# Speed vs FLOPs Scatter Plot (like paper)
plt.figure(figsize=(6,4))
plt.scatter(df["FLOPs"], df["Infer_ms"])
for i, row in df.iterrows():
    plt.text(row["FLOPs"], row["Infer_ms"], row["Type"])
plt.xlabel("FLOPs")
plt.ylabel("Inference Time (ms)")
plt.title("Speed vs FLOPs")
plt.savefig("/kaggle/working/speed_vs_flops.png")
plt.close()

In [ ]:
print("\nAll comparison plots saved to /kaggle/working/")


All comparison plots saved to /kaggle/working/
